In [ ]:
# Install necessary libraries
!pip install transformers[torch] datasets torch scikit-learn accelerate>=0.26.0 lollms_client onnxruntime 

In [ ]:
!pip install onnx

In [ ]:
random_state = 42
max_retry = 3
batch_size = 10  # Generate 10 examples at a time for data augmentation


# Fine-tuning TinyBERT for Multi-Class Text Classification
In this notebook, we will fine-tune a pre-trained BERT model for the text classification task in the Frugal AI Challenge. The steps include:
1. Loading the dataset.
2. Preprocessing the text data.
3. Adding a custom classification head to BERT.
4. Fine-tuning the model.
5. Evaluating the model's performance.


In [ ]:
# Import necessary libraries
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt


## Loading dataset

In [ ]:
# Load the dataset
dataset = load_dataset("QuotaClimat/frugalaichallenge-text-train", "default")

# Display the dataset structure
print(dataset)

# Split the training set into train and validation subsets
from sklearn.model_selection import train_test_split

# Convert the training set to a Pandas DataFrame for splitting
train_df = pd.DataFrame(dataset["train"])

# Drop the duplicate column "__index_level_0__" if it exists
if "__index_level_0__" in train_df.columns:
    train_df = train_df.drop(columns=["__index_level_0__"])

# Define the label text and organize them by their prefixes
label_texts = [
    "0_not_relevant",
    "1_not_happening",
    "2_not_human",
    "3_not_bad",
    "4_solutions_harmful_unnecessary",
    "5_science_unreliable",
    "6_proponents_biased",
    "7_fossil_fuels_needed"
]

# Sort the labels alphabetically by their prefix
sorted_labels = sorted(label_texts, key=lambda x: x.split("_")[0])

# Create a mapping from label text to integers based on the sorted order
label_mapping = {label: idx for idx, label in enumerate(sorted_labels)}

# Map the labels in the DataFrame
train_df["label"] = train_df["label"].map(label_mapping)

# Perform an 60-20-20 split for training, validation and test
train_data, test_data = train_test_split(train_df, test_size=0.2, stratify=train_df["label"], random_state=random_state)
train_data, val_data = train_test_split(train_data, test_size=0.1, stratify=train_data["label"], random_state=random_state)

# Convert the split data back to Hugging Face Dataset format
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

# Update the dataset dictionary to include the validation set
dataset = {
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
}

# Display the updated dataset structure
print(dataset)


# remove the index from the label:
label_mapping = {key.split("_", 1)[1]: value for key, value in label_mapping.items()}

# Display the label mapping for reference
print("Label Mapping:", label_mapping)


## Augmenting and balancing database
Here we use lollms to generate new examples to balance and augment the databse.

In [ ]:
from lollms_client import LollmsClient
from datasets import Dataset
import pandas as pd
import json
import random
from tqdm.notebook import tqdm

# Initialize LollmsClient
lc = LollmsClient("http://localhost:9600")

# Function to generate additional examples for a class
def generate_examples_for_class(class_name, existing_examples, num_examples_needed):
    generated_examples = []
    while num_examples_needed > 0:
        current_batch_size = min(batch_size, num_examples_needed)  # Adjust batch size for the remaining examples
        # Parse the generated JSON
        generated_data = None
        retry = 0
        while not generated_data and retry<max_retry:
            # Randomly select up to 10 examples from the existing examples
            random_examples = random.sample(existing_examples, min(len(existing_examples), 10))
            
            # Prepare the prompt with the randomly selected examples
            example_texts = ",\n".join([f'"{text}"' for text in random_examples])  # Use up to 10 random examples for the prompt
            
            # Build the JSON structure as a string
            json_structure = (
                "{\n"
                '    "class": "' + class_name + '",\n'
                '    "examples": [\n'
                + example_texts + "\n"
                "    ]\n"
                "}"
            )
            
            # Build the full prompt
            prompt_template = (
                "Build a JSON code that contains a list of new text examples in the same class: "
                + class_name
                + ".\nHere are some examples from the class:\n```json\n"
                + json_structure
                + "\n```\n\n"
                + "Generate "
                + str(batch_size)
                + " new examples in the same style and tone."
            )
            prompt = prompt_template.replace(str(batch_size), str(current_batch_size))  # Update batch size in the prompt
                        
            # Generate synthetic examples using LollmsClient
            response = lc.generate_code(prompt)
            if response:
                try:
                    generated_data = json.loads(response.strip())  # Parse the JSON response
                    if "examples" in generated_data:
                        generated_examples.extend(generated_data["examples"])
                        num_examples_needed -= len(generated_data["examples"])
                    else:
                        print(f"Unexpected response format: {response}")
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON response for class {class_name}: {e}")
                except KeyError as e:
                    print(f"KeyError in response for class {class_name}: {e}")
                except Exception as e:
                    print(f"Exception: {e}")
                retry+=1
        if retry>=max_retry:
            print("Warning: Max retries reached with no success")
    return generated_examples

# Function to augment and balance the dataset
def augment_and_balance_dataset(train_df, label_mapping, augmentation_coefficient=1.0):
    # Check class distribution
    class_counts = train_df["label"].value_counts()
    print("Class Distribution Before Augmentation and Balancing:")
    print(class_counts)

    # Determine the target number of examples per class
    target_count = int(class_counts.max() * augmentation_coefficient)

    # Generate additional examples for all classes
    new_data = []
    first = True
    total_classes = len(class_counts)
    for label, count in tqdm(class_counts.items(), desc="Augmenting and Balancing Classes", total=total_classes):
        class_name = [key for key, value in label_mapping.items() if value == label][0]  # Get the class name
        num_examples_needed = target_count - count
        existing_examples = train_df[train_df["label"] == label]["quote"].tolist()
        
        # Debug: Check existing examples
        if first:
            print(f"Class: {class_name}, Existing Examples: {len(existing_examples)}")
            first = False
        
        # Randomly select up to 10 examples
        random_examples = random.sample(existing_examples, min(len(existing_examples), 10))
        
        # Generate new examples
        generated_examples = generate_examples_for_class(class_name, random_examples, num_examples_needed)
        
        # Debug: Check generated examples
        print(f"Class: {class_name}, Needed: {num_examples_needed}, Generated: {len(generated_examples)}")
        
        # Add the generated examples to the new data
        for example in generated_examples:
            new_data.append({"quote": example, "label": label, "source": "lollms", "url":"", "language":"en", "subsource":"","id":0,'__index_level_0__':0})

    # Convert the new data to a DataFrame
    new_data_df = pd.DataFrame(new_data)

    # Debug: Check new data shape
    print(f"New Data Shape: {new_data_df.shape}")

    # Append the new data to the training DataFrame
    augmented_balanced_train_df = pd.concat([train_df, new_data_df], ignore_index=True)

    # Debug: Check final distribution
    print("Class Distribution After Augmentation and Balancing:")
    print(augmented_balanced_train_df["label"].value_counts())

    return augmented_balanced_train_df

# Example usage
# Assuming `train_df` is your training DataFrame and `label_mapping` is a dictionary mapping class names to labels
augmentation_coefficient = 1.1  # Example coefficient
augmented_balanced_train_df = augment_and_balance_dataset(train_df, label_mapping, augmentation_coefficient)

### Fix bad entries

In [ ]:
# This is a fix if some entries failed to be added correctly
# Function to process each entry in the 'quote' column
def process_quote_entry(entry):
    if isinstance(entry, dict):  # Check if the entry is a dictionary
        # Extract 'text' if it exists, otherwise 'example', otherwise 'Unknown'
        return entry.get('text') or entry.get('example') or "Unknown"
    return entry  # Leave non-dictionary entries unchanged

# Apply the function to the 'quote' column
augmented_balanced_train_df['quote'] = augmented_balanced_train_df['quote'].apply(process_quote_entry)

# Verify the changes
print(augmented_balanced_train_df['quote'].head())


### Verify the database

In [ ]:
from datasets import DatasetDict
# Determine the target number of examples per class (based on the majority class)
class_counts = train_df["label"].value_counts()
target_count = class_counts.max()

print(augmented_balanced_train_df)
# Convert the balanced DataFrame back to Hugging Face Dataset format
balanced_train_dataset = Dataset.from_pandas(augmented_balanced_train_df)

# Update the dataset dictionary
dataset["train"] = balanced_train_dataset
print(dataset)
balanced_dataset = DatasetDict(dataset)

# Check the new class distribution
balanced_class_counts = augmented_balanced_train_df["label"].value_counts()
print("Class Distribution After Balancing:")
print(balanced_class_counts)

In [ ]:
# Save the updated dataset to a local directory
balanced_dataset.save_to_disk("./data/balanced_dataset")

In [ ]:
from datasets import DatasetDict

# Load the dataset dictionary from the saved directory
dataset = DatasetDict.load_from_disk("./data/balanced_dataset")
print(dataset)

## Load TinyBERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=len(label_mapping))

# Tokenize the text data
def preprocess_data(examples):
    return tokenizer(examples["quote"], padding="max_length", truncation=True, max_length=128)

# Apply the tokenizer to the train and validation datasets
tokenized_train_dataset = dataset["train"].map(preprocess_data, batched=True)
tokenized_val_dataset = dataset["validation"].map(preprocess_data, batched=True)
tokenized_test_dataset = dataset["test"].map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


## Prepare training

In [ ]:
# Define a function to compute evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
# Define training arguments with gradient clipping
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    evaluation_strategy="epoch",    # Evaluate every epoch
    learning_rate=2e-5,             # Learning rate
    per_device_train_batch_size=16, # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    num_train_epochs=10,            # Increased epochs to allow early stopping
    weight_decay=0.01,              # Weight decay for regularization
    logging_dir="./logs",           # Directory for logs
    logging_steps=10,
    save_strategy="epoch",          # Save model at the end of each epoch
    load_best_model_at_end=True,    # Load the best model based on validation
    metric_for_best_model="eval_loss",  # Metric to monitor for best model
    greater_is_better=False,        # Lower eval_loss is better
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    max_grad_norm=1.0               # Gradient clipping
)


In [ ]:
from transformers import EarlyStoppingCallback

# Add EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,  # Stop after 2 epochs without improvement
    early_stopping_threshold=0.01  # Minimum improvement threshold
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]  # Add early stopping callback
)

# Train the model
trainer.train()


## Save the model 

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./models/TinyBert-fine-tuned")
tokenizer.save_pretrained("./models/TinyBert-fine-tuned")

In [ ]:
logs = pd.DataFrame(trainer.state.log_history)
print(logs)

## Build some metrics graphs

In [ ]:
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


# Get predictions on the validation set
predictions = trainer.predict(tokenized_test_dataset)

# Convert predictions to class labels
predicted_labels = predictions.predictions.argmax(axis=-1)
true_labels = predictions.label_ids

# Compute classification metrics
report = classification_report(true_labels, predicted_labels, target_names=list(label_mapping.keys()))
print(report)

# confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# Save the classification report to a text file
output_dir = Path("./models/TinyBert-fine-tuned")
output_dir.mkdir(parents=True, exist_ok=True)

classification_report_path = output_dir / "classification_report.txt"
with classification_report_path.open("w") as f:
    f.write(report)

# Extract training and validation metrics
epochs = logs["epoch"]
training_loss = logs["loss"]
validation_loss = logs["eval_loss"]
validation_accuracy = logs["eval_accuracy"]
training_accuracy = logs.get("accuracy", None)  # Ensure training accuracy is logged

# Plot Training and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs, training_loss, label="Training Loss", marker="o", linestyle="-")
plt.step(epochs, validation_loss, label="Validation Loss", marker="o", linestyle="--", where="post")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid()
loss_plot_path = output_dir / "training_loss_plot.png"
plt.savefig(loss_plot_path)

# Plot Validation Accuracy and Training Accuracy
plt.figure(figsize=(10, 5))
if training_accuracy:
    plt.plot(epochs, training_accuracy, label="Training Accuracy", marker="o", linestyle="-", color="blue")
plt.plot(epochs, validation_accuracy, label="Validation Accuracy", marker="o", linestyle="--", color="green")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.grid()
accuracy_plot_path = output_dir / "validation_accuracy_plot.png"
plt.savefig(accuracy_plot_path)

print(f"Plots saved to {output_dir}")



In [ ]:
# Save the confusion matrix plot
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=list(label_mapping.keys()), yticklabels=list(label_mapping.keys()))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.savefig("./models/TinyBert-fine-tuned/confusion_matrix.png")


In [ ]:
# Save the confusion matrix plot
print(conf_matrix)

## Send to hugging face

In [ ]:
!huggingface-cli login

In [ ]:
def generate_model_card(
    model_dir, model_name, description, metrics_data, limitations, citation, label_mapping
):
    """
    Generates a model card and saves it as README.md in the specified directory.

    Args:
        model_dir (str): Directory where the model card will be saved.
        model_name (str): Name of the model.
        description (str): Description of the model.
        metrics_data (dict): Performance metrics of the model (e.g., precision, recall, F1-score, accuracy).
        limitations (str): Limitations of the model.
        citation (str): Citation for the model.
        label_mapping (dict): Mapping between model output indices and class names.
    """
    # Generate the metrics table dynamically
    metrics_table = "| Class | Precision | Recall | F1-Score | Support |\n"
    metrics_table += "|-------|-----------|--------|----------|---------|\n"
    print(metrics_data)
    for idx, class_name in label_mapping.items():
        metrics_table += f"| {class_name} | {metrics_data['precision'][idx]:.2f} | {metrics_data['recall'][idx]:.2f} | {metrics_data['f1'][idx]:.2f} | {metrics_data['support'][idx]} |\n"

    # Add overall metrics
    overall_metrics = (
        f"- **Overall Accuracy**: {metrics_data['accuracy']:.2f}\n"
        f"- **Macro Average**: Precision: {metrics_data['macro_precision']:.2f}, Recall: {metrics_data['macro_recall']:.2f}, F1-Score: {metrics_data['macro_f1']:.2f}\n"
        f"- **Weighted Average**: Precision: {metrics_data['weighted_precision']:.2f}, Recall: {metrics_data['weighted_recall']:.2f}, F1-Score: {metrics_data['weighted_f1']:.2f}\n"
    )

    # Generate the model card content
    model_card_content = f"""
---
license: apache-2.0
datasets:
- QuotaClimat/frugalaichallenge-text-train
language:
- en
metrics:
- accuracy
- f1
base_model:
- huawei-noah/TinyBERT_General_4L_312D
library_name: transformers
---

# Model Card: {model_name}

## Model Overview
{description}

## Dataset
- **Source**: Frugal AI Challenge Text Task Dataset
- **Classes**: {len(label_mapping)} unique labels representing various categories of text
- **Preprocessing**: Tokenization using `BertTokenizer` with padding and truncation to a maximum sequence length of 128.

## Model Architecture
- **Base Model**: `huawei-noah/TinyBERT_General_4L_312D`
- **Classification Head**: cross-entropy loss.
- **Number of Labels**: {len(label_mapping)}

## Training Details
- **Optimizer**: AdamW
- **Learning Rate**: 2e-5
- **Batch Size**: 16 (for both training and evaluation)
- **Epochs**: 3
- **Weight Decay**: 0.01
- **Evaluation Strategy**: Performed at the end of each epoch
- **Hardware**: Trained on GPUs for efficient computation

## Performance Metrics (Validation Set)
The following metrics were computed on the validation set (not the test set, which remains private for the competition):

{metrics_table}

{overall_metrics}

## Training Evolution
### Training and Validation Loss
The training and validation loss evolution over epochs is shown below:

![Training Loss](./training_loss_plot.png)

### Validation Accuracy
The validation accuracy evolution over epochs is shown below:

![Validation Accuracy](./validation_accuracy_plot.png)

## Confusion Matrix
The confusion matrix below illustrates the model's performance on the validation set, highlighting areas of strength and potential misclassifications:

![Confusion Matrix](./confusion_matrix.png)

## Key Features
- **Class Weighting**: Addressed dataset imbalance by incorporating class weights during training.
- **Custom Loss Function**: Used weighted cross-entropy loss for better handling of underrepresented classes.
- **Evaluation Metrics**: Accuracy, precision, recall, and F1-score were computed to provide a comprehensive understanding of the model's performance.

## Class Mapping
The mapping between model output indices and class names is as follows:
{', '.join([f"{idx}: {class_name}" for idx, class_name in label_mapping.items()])}

## Usage
This model can be used for multi-class text classification tasks where the input text needs to be categorized into one of the eight predefined classes. It is particularly suited for datasets with class imbalance, thanks to its weighted loss function.

### Example Usage
```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("{model_name}")
tokenizer = AutoTokenizer.from_pretrained("{model_name}")

# Tokenize input text
text = "Your input text here"
inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

# Perform inference
outputs = model(**inputs)
predicted_class = outputs.logits.argmax(-1).item()

print(f"Predicted Class: {{predicted_class}}")
```

## Limitations
{limitations}

## Citation
{citation}

## Acknowledgments
Special thanks to the Frugal AI Challenge organizers for providing the dataset and fostering innovation in AI research.
    """
    # Save the model card as README.md
    with open(f"{model_dir}/README.md", "w") as f:
        f.write(model_card_content)


In [ ]:
from pathlib import Path
from sklearn.metrics import classification_report
import numpy as np

# Get predictions for the validation set
val_predictions = trainer.predict(tokenized_val_dataset)
preds = np.argmax(val_predictions.predictions, axis=-1)
labels = val_predictions.label_ids

# Get detailed classification report
report = classification_report(labels, preds, output_dict=True)

# Create metrics data using the available results
metrics_data = {
    'precision': {
        0: report['0']['precision'],
        1: report['1']['precision'],
        2: report['2']['precision'],
        3: report['3']['precision'],
        4: report['4']['precision'],
        5: report['5']['precision'],
        6: report['6']['precision'],
        7: report['7']['precision']
    },
    'recall': {
        0: report['0']['recall'],
        1: report['1']['recall'],
        2: report['2']['recall'],
        3: report['3']['recall'],
        4: report['4']['recall'],
        5: report['5']['recall'],
        6: report['6']['recall'],
        7: report['7']['recall']
    },
    'f1': {
        0: report['0']['f1-score'],
        1: report['1']['f1-score'],
        2: report['2']['f1-score'],
        3: report['3']['f1-score'],
        4: report['4']['f1-score'],
        5: report['5']['f1-score'],
        6: report['6']['f1-score'],
        7: report['7']['f1-score']
    },
    'support': {
        0: report['0']['support'],
        1: report['1']['support'],
        2: report['2']['support'],
        3: report['3']['support'],
        4: report['4']['support'],
        5: report['5']['support'],
        6: report['6']['support'],
        7: report['7']['support']
    },
    'accuracy': report['accuracy'],
    'macro_precision': report['macro avg']['precision'],
    'macro_recall': report['macro avg']['recall'],
    'macro_f1': report['macro avg']['f1-score'],
    'weighted_precision': report['weighted avg']['precision'],
    'weighted_recall': report['weighted avg']['recall'],
    'weighted_f1': report['weighted avg']['f1-score']
}

# Updated label mapping with actual classes
label_mapping = {
    0: "not_relevant",
    1: "not_happening",
    2: "not_human",
    3: "not_bad",
    4: "solutions_harmful_unnecessary",
    5: "science_unreliable",
    6: "proponents_biased",
    4: "fossil_fuels_needed"
}

model_dir = Path("./models/TinyBert-fine-tuned")

generate_model_card(
    model_dir=model_dir,
    model_name="climate-skepticism-classifier",
    description="""This model implements a novel approach to classifying climate change skepticism arguments 
    by utilizing Large Language Models (LLMs) for data rebalancing. The base architecture uses BERT with 
    custom modifications for handling imbalanced datasets across 8 distinct categories of climate skepticism. 
    The model achieves exceptional performance with an accuracy of 99.92%.

    The model categorizes text into the following skepticism types:
    - Fossil fuel necessity arguments
    - Non-relevance claims
    - Climate change denial
    - Anthropogenic cause denial
    - Impact minimization
    - Bias allegations
    - Scientific reliability questions
    - Solution opposition
    
    The unique feature of this model is its use of LLM-based data rebalancing to address the inherent class 
    imbalance in climate skepticism detection, ensuring robust performance across all argument categories.""",
    metrics_data=metrics_data,
    limitations="""- Performance may vary on extremely imbalanced datasets
    - Requires significant computational resources for training
    - Model performance is dependent on the quality of LLM-generated balanced data
    - May not perform optimally on very long text sequences (>128 tokens)
    - May struggle with novel or evolving climate skepticism arguments
    - Could be sensitive to subtle variations in argument framing
    - May require periodic updates to capture emerging skepticism patterns""",
    citation="""If you use this model, please cite:
    @article{your_name2024climateskepticism,
        title={LLM-Rebalanced Transformer for Climate Change Skepticism Classification},
        author={Your Name},
        year={2024},
        journal={Preprint}
    }""",
    label_mapping=label_mapping
)


In [ ]:
print(metrics_data)

## Convert to ONNX

In [ ]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import torch
# Load the fine-tuned model and tokenizer
model_path = "./TinyBert-fine-tuned"
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Define the ONNX export path
onnx_path = Path("TinyBert-fine-tuned.onnx")

# Dummy input for tracing
dummy_input = tokenizer("This is a sample input", return_tensors="pt")

# Export the model to ONNX
torch.onnx.export(
    model,
    (dummy_input["input_ids"], dummy_input["attention_mask"]),
    onnx_path,
    input_names=["input_ids", "attention_mask"],
    output_names=["output"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "output": {0: "batch_size", 1: "sequence_length"},
    },
    opset_version=14,  # Ensure compatibility with ONNX runtime
)
print(f"Model exported to {onnx_path}")


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./models/TinyBert-fine-tuned")
tokenizer = AutoTokenizer.from_pretrained("./models/TinyBert-fine-tuned")

# Push the model to Hugging Face
model.push_to_hub("ParisNeo/TinyBert-frugal-ai-text-classification")
tokenizer.push_to_hub("ParisNeo/TinyBert-frugal-ai-text-classification")
